In [1]:
import os
import copy
import tqdm
import pickle
import random
import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

### Data Wrangling ###
import numpy as np
import pandas as pd
from scipy import stats

### Machine Learning ###
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer



from pickle import load,dump

### Make prettier the prints ###
from colorama import Fore
c_ = Fore.CYAN
m_ = Fore.MAGENTA
r_ = Fore.RED
b_ = Fore.BLUE
y_ = Fore.YELLOW
g_ = Fore.GREEN

import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
y_train_non = train_targets_nonscored.copy()

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
n_comp_GENES = 463
n_comp_CELLS = 60
VarianceThreshold_for_FS = 0.9
Dropout_Model = 0.25
print('n_comp_GENES', n_comp_GENES, 'n_comp_CELLS', n_comp_CELLS, 'total', n_comp_GENES + n_comp_CELLS)

n_comp_GENES 463 n_comp_CELLS 60 total 523


In [5]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [6]:
for col in (GENES + CELLS):    
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")   # from optimal commit 9
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(seed=42)

In [8]:
data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
pca_g=PCA(n_components=n_comp_GENES, random_state=42)
data2 = (pca_g.fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_GENES)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [9]:
data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
pca_c=PCA(n_components=n_comp_CELLS, random_state=42)
data2 = (pca_c.fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_CELLS)])

train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [10]:
data = train_features.append(test_features)

In [11]:
var_thresh = VarianceThreshold(VarianceThreshold_for_FS)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(23814, 1014)

In [12]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [13]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [14]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [15]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,0,5
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,0,6
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,-0.150978,...,0,0,0,0,0,0,0,0,0,5
21944,id_fffb1ceed,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,0.735603,...,0,0,0,0,0,0,0,0,0,1
21945,id_fffb70c0c,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,-0.304313,...,0,0,0,0,0,0,0,0,0,5
21946,id_fffcb9e7c,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,0.048942,...,0,0,0,0,0,0,0,0,0,1


In [16]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1219)
(21948, 1220)
(3624, 1013)
(21948, 207)
(3982, 207)


In [17]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

1015

In [18]:
def get_model(hp):
    
    model=tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(1015,)))
    model.add(tf.keras.layers.BatchNormalization())

    for i in range(hp.Int("N_Layers",2,6)):
    #         model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(hp.Int(f"number_of_neurons1_{i}",512,2048), activation=hp.Choice(f"activation1_{i}",values=['elu', 'swish', 'selu']))))
        model.add(tf.keras.layers.Dense(hp.Int(f"number_of_neurons2_{i}",512,2048), activation=hp.Choice(f"activation2_{i}",values=['elu', 'swish', 'selu'])))
        model.add(tf.keras.layers.Dropout(hp.Float(f"Dropout_{i}",0.0,0.7,step=0.01)))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(206,activation='sigmoid'))


    model.compile(optimizer = tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
#     model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    
    return model

In [19]:
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import kerastuner as kt

In [20]:
X_train=process_data(folds).loc[:,feature_cols]
y_train=process_data(folds).loc[:,target_cols]

In [21]:
y_train_non = y_train_non.drop(['sig_id'],1)

In [22]:
y_train

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21944,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21945,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# y_train_non=y_train_non.iloc[:,y_train_non.columns!='sig_id']
final_y=pd.concat([y_train],1)

In [24]:
models=[]
hps=[]

for seed in [30,58,564]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        
        tuner=BayesianOptimization(get_model,
                     kt.Objective("val_loss", direction="min"),
                     max_trials = 4, overwrite=True) 
        
        tuner.search(X_train.values[tr]
                     ,final_y.values[tr]
                     ,epochs=150
                     ,batch_size=128
                     ,validation_data=(X_train.values[te],final_y.values[te])
                     ,verbose=1,callbacks=[
              tf.keras.callbacks.ReduceLROnPlateau(
                  monitor='val_loss', 
                  factor=0.1, 
                  patience=3,
                  epsilon = 1e-4, 
                  mode = 'min',
                  verbose=1
              )
              ,
              
              tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss',
                  min_delta=0,
                  patience=10,
                  mode='auto',
                  verbose=1,
                  baseline=None,
                  restore_best_weights=True
              )
          ])
        
        hps.append(tuner.get_best_hyperparameters(num_trials = 1)[0])
        models.append(tuner.get_best_models(num_models=1)[0])


# for seed in [30,58,564]:
# # for seed in [30]:
#     for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
#         print("SEED:"+str(seed)+" Fold:"+str(n))
        
#         tuner=BayesianOptimization(get_model,
#                      kt.Objective("val_loss", direction="min"),
#                      max_trials = 4, overwrite=True) 
        
#         tuner.search(X_train.values[tr]
#                      ,y_train.values[tr]
#                      ,epochs=150
#                      ,batch_size=128
#                      ,validation_data=(X_train.values[te],y_train.values[te])
#                      ,verbose=1,callbacks=[
#               tf.keras.callbacks.ReduceLROnPlateau(
#                   monitor='val_loss', 
#                   factor=0.1, 
#                   patience=3,
#                   epsilon = 1e-4, 
#                   mode = 'min',
#                   verbose=1
#               )
#               ,
              
#               tf.keras.callbacks.EarlyStopping(
#                   monitor='val_loss',
#                   min_delta=0,
#                   patience=10,
#                   mode='auto',
#                   verbose=1,
#                   baseline=None,
#                   restore_best_weights=True
#               )
#           ])
        
#         hps.append(tuner.get_best_hyperparameters(num_trials = 1)[0])
#         models.append(tuner.get_best_models(num_models=1)[0])

SEED:30 Fold:0
Epoch 1/150
138/138 [==============================] - 10s 76ms/step - loss: 0.2941 - accuracy: 0.0412 - val_loss: 0.0328 - val_accuracy: 0.0745
Epoch 2/150
138/138 [==============================] - 10s 74ms/step - loss: 0.0230 - accuracy: 0.0758 - val_loss: 0.0204 - val_accuracy: 0.0841
Epoch 3/150
138/138 [==============================] - 10s 74ms/step - loss: 0.0191 - accuracy: 0.1036 - val_loss: 0.0181 - val_accuracy: 0.1319
Epoch 4/150
138/138 [==============================] - 10s 72ms/step - loss: 0.0177 - accuracy: 0.1252 - val_loss: 0.0175 - val_accuracy: 0.1146
Epoch 5/150
138/138 [==============================] - 10s 74ms/step - loss: 0.0168 - accuracy: 0.1463 - val_loss: 0.0173 - val_accuracy: 0.1269
Epoch 6/150
138/138 [==============================] - 11s 79ms/step - loss: 0.0161 - accuracy: 0.1588 - val_loss: 0.0170 - val_accuracy: 0.1335
Epoch 7/150
138/138 [==============================] - 10s 71ms/step - loss: 0.0152 - accuracy: 0.1821 - val_loss: 

Epoch 1/150
138/138 [==============================] - 10s 71ms/step - loss: 0.3172 - accuracy: 0.0238 - val_loss: 0.0301 - val_accuracy: 0.0203
Epoch 2/150
138/138 [==============================] - 11s 78ms/step - loss: 0.0259 - accuracy: 0.0510 - val_loss: 0.0212 - val_accuracy: 0.0802
Epoch 3/150
138/138 [==============================] - 12s 85ms/step - loss: 0.0215 - accuracy: 0.0579 - val_loss: 0.0198 - val_accuracy: 0.0549
Epoch 4/150
138/138 [==============================] - 10s 71ms/step - loss: 0.0204 - accuracy: 0.0710 - val_loss: 0.0190 - val_accuracy: 0.0759
Epoch 5/150
138/138 [==============================] - 10s 70ms/step - loss: 0.0198 - accuracy: 0.0881 - val_loss: 0.0184 - val_accuracy: 0.0882
Epoch 6/150
138/138 [==============================] - 10s 73ms/step - loss: 0.0191 - accuracy: 0.0993 - val_loss: 0.0179 - val_accuracy: 0.0966
Epoch 7/150
138/138 [==============================] - 10s 71ms/step - loss: 0.0187 - accuracy: 0.1048 - val_loss: 0.0178 - val_ac

Epoch 1/150
138/138 [==============================] - 8s 57ms/step - loss: 0.2670 - accuracy: 0.0282 - val_loss: 0.0273 - val_accuracy: 0.0743
Epoch 2/150
138/138 [==============================] - 8s 57ms/step - loss: 0.0240 - accuracy: 0.0521 - val_loss: 0.0208 - val_accuracy: 0.0535
Epoch 3/150
138/138 [==============================] - 8s 58ms/step - loss: 0.0220 - accuracy: 0.0733 - val_loss: 0.0197 - val_accuracy: 0.0768
Epoch 4/150
138/138 [==============================] - 8s 59ms/step - loss: 0.0205 - accuracy: 0.0878 - val_loss: 0.0187 - val_accuracy: 0.1255
Epoch 5/150
138/138 [==============================] - 9s 68ms/step - loss: 0.0195 - accuracy: 0.1020 - val_loss: 0.0182 - val_accuracy: 0.0927
Epoch 6/150
138/138 [==============================] - 8s 61ms/step - loss: 0.0189 - accuracy: 0.1129 - val_loss: 0.0176 - val_accuracy: 0.1062
Epoch 7/150
138/138 [==============================] - 8s 57ms/step - loss: 0.0184 - accuracy: 0.1187 - val_loss: 0.0174 - val_accuracy:

Epoch 1/150
138/138 [==============================] - 9s 63ms/step - loss: 0.2553 - accuracy: 0.0290 - val_loss: 0.0234 - val_accuracy: 0.0629
Epoch 2/150
138/138 [==============================] - 9s 63ms/step - loss: 0.0237 - accuracy: 0.0555 - val_loss: 0.0213 - val_accuracy: 0.0601
Epoch 3/150
138/138 [==============================] - 8s 59ms/step - loss: 0.0216 - accuracy: 0.0716 - val_loss: 0.0196 - val_accuracy: 0.1171
Epoch 4/150
138/138 [==============================] - 9s 62ms/step - loss: 0.0205 - accuracy: 0.0918 - val_loss: 0.0189 - val_accuracy: 0.0856
Epoch 5/150
138/138 [==============================] - 8s 57ms/step - loss: 0.0197 - accuracy: 0.1011 - val_loss: 0.0181 - val_accuracy: 0.1021
Epoch 6/150
138/138 [==============================] - 9s 65ms/step - loss: 0.0189 - accuracy: 0.1099 - val_loss: 0.0177 - val_accuracy: 0.1059
Epoch 7/150
138/138 [==============================] - 9s 66ms/step - loss: 0.0188 - accuracy: 0.1207 - val_loss: 0.0175 - val_accuracy:

SEED:30 Fold:1
Epoch 1/150
138/138 [==============================] - 8s 61ms/step - loss: 0.2997 - accuracy: 0.0296 - val_loss: 0.0291 - val_accuracy: 0.0410
Epoch 2/150
138/138 [==============================] - 8s 58ms/step - loss: 0.0231 - accuracy: 0.0773 - val_loss: 0.0204 - val_accuracy: 0.0847
Epoch 3/150
138/138 [==============================] - 8s 59ms/step - loss: 0.0194 - accuracy: 0.1077 - val_loss: 0.0186 - val_accuracy: 0.1364
Epoch 4/150
138/138 [==============================] - 8s 57ms/step - loss: 0.0181 - accuracy: 0.1259 - val_loss: 0.0179 - val_accuracy: 0.1173
Epoch 5/150
138/138 [==============================] - 8s 60ms/step - loss: 0.0172 - accuracy: 0.1365 - val_loss: 0.0175 - val_accuracy: 0.1271
Epoch 6/150
138/138 [==============================] - 6s 46ms/step - loss: 0.0167 - accuracy: 0.1460 - val_loss: 0.0171 - val_accuracy: 0.1342
Epoch 7/150
138/138 [==============================] - 7s 48ms/step - loss: 0.0160 - accuracy: 0.1601 - val_loss: 0.0169 

...

Epoch 1/150
138/138 [==============================] - 14s 103ms/step - loss: 0.2850 - accuracy: 0.0247 - val_loss: 0.0275 - val_accuracy: 0.0638
Epoch 2/150
138/138 [==============================] - 14s 105ms/step - loss: 0.0243 - accuracy: 0.0522 - val_loss: 0.0206 - val_accuracy: 0.0936
Epoch 3/150
138/138 [==============================] - 14s 101ms/step - loss: 0.0216 - accuracy: 0.0755 - val_loss: 0.0194 - val_accuracy: 0.0875
Epoch 4/150
138/138 [==============================] - 14s 101ms/step - loss: 0.0203 - accuracy: 0.0918 - val_loss: 0.0186 - val_accuracy: 0.1005
Epoch 5/150
138/138 [==============================] - 14s 103ms/step - loss: 0.0194 - accuracy: 0.1113 - val_loss: 0.0183 - val_accuracy: 0.1036
Epoch 6/150
138/138 [==============================] - 15s 106ms/step - loss: 0.0186 - accuracy: 0.1217 - val_loss: 0.0177 - val_accuracy: 0.1123
Epoch 7/150
138/138 [==============================] - 14s 99ms/step - loss: 0.0182 - accuracy: 0.1247 - val_loss: 0.0174 - 

SEED:564 Fold:4
Epoch 1/150
138/138 [==============================] - 8s 54ms/step - loss: 0.2955 - accuracy: 0.0334 - val_loss: 0.0308 - val_accuracy: 0.0342
Epoch 2/150
138/138 [==============================] - 7s 53ms/step - loss: 0.0228 - accuracy: 0.0834 - val_loss: 0.0206 - val_accuracy: 0.0952
Epoch 3/150
138/138 [==============================] - 8s 55ms/step - loss: 0.0188 - accuracy: 0.1196 - val_loss: 0.0181 - val_accuracy: 0.1426
Epoch 4/150
138/138 [==============================] - 8s 57ms/step - loss: 0.0174 - accuracy: 0.1376 - val_loss: 0.0173 - val_accuracy: 0.1285
Epoch 5/150
138/138 [==============================] - 7s 53ms/step - loss: 0.0162 - accuracy: 0.1650 - val_loss: 0.0170 - val_accuracy: 0.1372
Epoch 6/150
138/138 [==============================] - 7s 52ms/step - loss: 0.0150 - accuracy: 0.1960 - val_loss: 0.0173 - val_accuracy: 0.1394
Epoch 7/150
138/138 [==============================] - 7s 54ms/step - loss: 0.0137 - accuracy: 0.2282 - val_loss: 0.0175

Epoch 1/150
138/138 [==============================] - 12s 85ms/step - loss: 0.3025 - accuracy: 0.0223 - val_loss: 0.0337 - val_accuracy: 0.0226
Epoch 2/150
138/138 [==============================] - 12s 84ms/step - loss: 0.0235 - accuracy: 0.0552 - val_loss: 0.0209 - val_accuracy: 0.0638
Epoch 3/150
138/138 [==============================] - 12s 89ms/step - loss: 0.0199 - accuracy: 0.0851 - val_loss: 0.0190 - val_accuracy: 0.0930
Epoch 4/150
138/138 [==============================] - 12s 85ms/step - loss: 0.0186 - accuracy: 0.1091 - val_loss: 0.0180 - val_accuracy: 0.0987
Epoch 5/150
138/138 [==============================] - 12s 87ms/step - loss: 0.0177 - accuracy: 0.1187 - val_loss: 0.0175 - val_accuracy: 0.1130
Epoch 6/150
138/138 [==============================] - 12s 86ms/step - loss: 0.0171 - accuracy: 0.1396 - val_loss: 0.0173 - val_accuracy: 0.1310
Epoch 7/150
138/138 [==============================] - 12s 84ms/step - loss: 0.0165 - accuracy: 0.1477 - val_loss: 0.0171 - val_ac

Epoch 1/150
138/138 [==============================] - 16s 115ms/step - loss: 0.2972 - accuracy: 0.0237 - val_loss: 0.0300 - val_accuracy: 0.0337
Epoch 2/150
138/138 [==============================] - 16s 117ms/step - loss: 0.0247 - accuracy: 0.0511 - val_loss: 0.0207 - val_accuracy: 0.0864
Epoch 3/150
138/138 [==============================] - 16s 114ms/step - loss: 0.0218 - accuracy: 0.0667 - val_loss: 0.0200 - val_accuracy: 0.1066
Epoch 4/150
138/138 [==============================] - 16s 114ms/step - loss: 0.0202 - accuracy: 0.0887 - val_loss: 0.0185 - val_accuracy: 0.1016
Epoch 5/150
138/138 [==============================] - 16s 116ms/step - loss: 0.0191 - accuracy: 0.1079 - val_loss: 0.0180 - val_accuracy: 0.1062
Epoch 6/150
138/138 [==============================] - 16s 117ms/step - loss: 0.0183 - accuracy: 0.1185 - val_loss: 0.0176 - val_accuracy: 0.1069
Epoch 7/150
138/138 [==============================] - 16s 118ms/step - loss: 0.0180 - accuracy: 0.1234 - val_loss: 0.0173 -

Epoch 1/150
138/138 [==============================] - 19s 141ms/step - loss: 0.2658 - accuracy: 0.0300 - val_loss: 0.0272 - val_accuracy: 0.0560
Epoch 2/150
138/138 [==============================] - 20s 142ms/step - loss: 0.0225 - accuracy: 0.0592 - val_loss: 0.0203 - val_accuracy: 0.0695
Epoch 3/150
138/138 [==============================] - 19s 138ms/step - loss: 0.0198 - accuracy: 0.0938 - val_loss: 0.0185 - val_accuracy: 0.0957
Epoch 4/150
138/138 [==============================] - 19s 136ms/step - loss: 0.0184 - accuracy: 0.1128 - val_loss: 0.0177 - val_accuracy: 0.1064
Epoch 5/150
138/138 [==============================] - 20s 144ms/step - loss: 0.0177 - accuracy: 0.1297 - val_loss: 0.0175 - val_accuracy: 0.1212
Epoch 6/150
138/138 [==============================] - 19s 139ms/step - loss: 0.0169 - accuracy: 0.1400 - val_loss: 0.0173 - val_accuracy: 0.1169
Epoch 7/150
138/138 [==============================] - 19s 137ms/step - loss: 0.0165 - accuracy: 0.1531 - val_loss: 0.0185 -

In [25]:
# tuner.search(X_train,y_train,epochs=150,batch_size=128,validation_split=0.2,verbose=1,callbacks=[
#               tf.keras.callbacks.ReduceLROnPlateau(
#                   monitor='val_loss', 
#                   factor=0.1, 
#                   patience=3,
#                   epsilon = 1e-4, 
#                   mode = 'min',
#                   verbose=1
#               )
#               ,
              
#               tf.keras.callbacks.EarlyStopping(
#                   monitor='val_loss',
#                   min_delta=0,
#                   patience=10,
#                   mode='auto',
#                   verbose=1,
#                   baseline=None,
#                   restore_best_weights=True
#               )
#           ])


In [26]:
# hps=tuner.get_best_hyperparameters(num_trials = 10)[:10]
# models=tuner.get_best_models(num_models=10)[:10]

for i,(best_hps,model) in enumerate(zip(hps,models)):
    print("===================================================")
    print(f"{i}:")
#     print(f"Learning Rate: {best_hps.get('learning_rate')}")
#     print(f"Weight Decay: {best_hps.get('weight_decay')}")
    print(f"Number of Layers:{best_hps.get('N_Layers')}")
    for i in range(best_hps.get('N_Layers')):
        print(f"Dropout_{i} : "+str(best_hps.get(f"Dropout_{i}")))
#         print(f"number_of_neurons1_{i} : "+str(best_hps.get(f"number_of_neurons1_{i}")))
        print(f"number_of_neurons2_{i} : "+str(best_hps.get(f"number_of_neurons2_{i}")))
#         print(f"activation1_{i} : "+str(best_hps.get(f"activation1_{i}")))
        print(f"activation2_{i} : "+str(best_hps.get(f"activation2_{i}")))
        print()
    model.summary()

0:
Number of Layers:3
Dropout_0 : 0.63
number_of_neurons2_0 : 881
activation2_0 : elu

Dropout_1 : 0.6
number_of_neurons2_1 : 677
activation2_1 : selu

Dropout_2 : 0.28
number_of_neurons2_2 : 2048
activation2_2 : swish

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1015)              4060      
_________________________________________________________________
dense (Dense)                (None, 881)               895096    
_________________________________________________________________
dropout (Dropout)            (None, 881)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 881)               3524      
_________________________________________________________________
dense_1 (Dense)              (None, 677)               597114    
__________________________________

In [27]:
a=0
params={}
for i,seed in enumerate([30,58,564]):
    seeddata=[]
    for fold in range(5):
        print(a,seed,fold)
        model=models[a]
        best_hps=hps[a]
        
        droupouts=[]
        units=[]
        activations=[]
        for j in range(best_hps.get('N_Layers')):
            droupouts.append(best_hps.get(f"Dropout_{j}"))
            units.append(best_hps.get(f"number_of_neurons2_{j}"))
            activations.append(best_hps.get(f"activation2_{j}"))
        
        seeddata.append({
            "units":units,
            "activations":activations,
            "dropouts":droupouts
        })
        a+=1
    params[seed]=seeddata

0 30 0
1 30 1
2 30 2
3 30 3
4 30 4
5 58 0
6 58 1
7 58 2
8 58 3
9 58 4
10 564 0
11 564 1
12 564 2
13 564 3
14 564 4


In [28]:
params

{30: [{'units': [881, 677, 2048],
   'activations': ['elu', 'selu', 'swish'],
   'dropouts': [0.63, 0.6, 0.28]},
  {'units': [927, 1252, 512],
   'activations': ['swish', 'selu', 'elu'],
   'dropouts': [0.65, 0.15, 0.0]},
  {'units': [1089, 975, 770, 904, 1012, 985],
   'activations': ['swish', 'elu', 'elu', 'selu', 'elu', 'elu'],
   'dropouts': [0.43, 0.61, 0.12, 0.48, 0.26, 0.59]},
  {'units': [583, 1082, 1968],
   'activations': ['swish', 'swish', 'selu'],
   'dropouts': [0.38, 0.51, 0.49]},
  {'units': [1116, 623, 512, 512],
   'activations': ['selu', 'elu', 'elu', 'elu'],
   'dropouts': [0.53, 0.53, 0.0, 0.0]}],
 58: [{'units': [1911, 1251, 625, 512, 512, 512],
   'activations': ['elu', 'swish', 'selu', 'elu', 'elu', 'elu'],
   'dropouts': [0.5700000000000001, 0.13, 0.37, 0.0, 0.0, 0.0]},
  {'units': [1605, 998, 1176],
   'activations': ['swish', 'selu', 'elu'],
   'dropouts': [0.5700000000000001, 0.51, 0.61]},
  {'units': [1608, 512, 2045],
   'activations': ['selu', 'swish', 'el